In [74]:
from typing import List

import pandas as pd
import json
import altair as alt
from hun_date_parser import text2datetime
from hun_date_parser.date_parser.interval_restriction import extract_datetime_within_interval
from datetimerange import DateTimeRange
from copy import deepcopy
from datetime import datetime
from enum import Enum

class RequestFeedback:
    REQUEST_OK = "REQUEST_OK"
    REQUEST_SKIPPED = "REQUEST_SKIPPED"


def has_date_mention(text):
    res = text2datetime(text)
    return bool(res)


class TimeTable:
    """
    Time interval state representation
    """

    def __init__(self, labels: List[str]):
        """
        :param
        """
        self.labels = labels
        self.sub_datetimes = {l: [] for l in labels}
        self.current_dtrl = {'label': None, 'ladder': None}
        self.has_currently_discussed_range = False

    def _house_keeping(self):
        for label in self.labels:
            for i, dtr_i in enumerate(self.sub_datetimes[label]):
                for j, dtr_j in enumerate(self.sub_datetimes[label]):
                    if i < j and dtr_i.is_intersection(dtr_j):
                        self.sub_datetimes[label][i] = self.sub_datetimes[label][i].encompass(
                            self.sub_datetimes[label][j])
                        del self.sub_datetimes[label][j]
                        self._house_keeping()
                        break

    def label_timerange(self, start, end, label):
        new_dtr = DateTimeRange(start, end)

        encompassed = False
        for i in range(len(self.sub_datetimes[label])):
            if self.sub_datetimes[label][i].is_intersection(new_dtr):
                encompassed = True
                self.sub_datetimes[label][i] = self.sub_datetimes[label][i].encompass(new_dtr)

        if not encompassed:
            self.sub_datetimes[label].append(new_dtr)

        self._house_keeping()

    def query_timerange(self, start, end, label):
        queried_dtr = DateTimeRange(start, end)
        result = []

        for dtr in self.sub_datetimes[label]:
            if queried_dtr.is_intersection(dtr):
                result.append(queried_dtr.intersection(dtr))

        return result

    def label_timerange_by_text(self, text, label):
        res_l = text2datetime(text)

        for tval in res_l:
            if tval['start_date'] and tval['end_date']:
                self.label_timerange(tval['start_date'], tval['end_date'], label)

    def flush_label(self, label):
        self.sub_datetimes[label] = []

    def get_label(self, label):
        return self.sub_datetimes[label]

    def get_first_range_for_label(self, label):
        if not self.sub_datetimes[label]:
            return None

        min_dtr = self.sub_datetimes[label][0]
        for dtrange in self.sub_datetimes[label][1:]:
            if dtrange.start_datetime < min_dtr.start_datetime:
                min_dtr = dtrange

        return min_dtr
    
    def set_current_discussed(self, top_range_dct, label):
        top_range = DateTimeRange(top_range_dct['start_date'], top_range_dct['end_date'])
        dtrl = DateRangeLadder(top_range)
        self.current_dtrl = {'label': label, 'ladder': dtrl}
        self.has_currently_discussed_range = True
        
    def discuss_current(self, query_text):
        if not self.current_dtrl['ladder'] or not has_date_mention(query_text):
            return RequestFeedback.REQUEST_SKIPPED
        
        self.current_dtrl['ladder'].step_in_ladder_with_text(query_text)
        if not self.current_dtrl['ladder'].has_range():
            self.has_currently_discussed_range = False
            
        return RequestFeedback.REQUEST_OK
            
    def get_currently_discussed_range(self):
        if self.has_currently_discussed_range:
            return self.current_dtrl['ladder'].get_bottom_step()
        else:
            return RequestFeedback.REQUEST_SKIPPED
        
    def remove_currently_discussed(self):
        self.current_dtrl = {'label': None, 'ladder': None}
        self.has_currently_discussed_range = False

    def get_viz(self):
        res = []

        for label, ti_list in self.sub_datetimes.items():
            for dtr in ti_list:
                res.append([label, dtr.start_datetime, dtr.end_datetime])

        df_timetable = pd.DataFrame(res, columns=['label', 'from', 'to'])

        alt.renderers.enable('default')
        alt.Chart(df_timetable).mark_bar().encode(
            x='from',
            x2='to',
            y='label',
            tooltip=[alt.Tooltip('from', format='%Y-%m-%d %H:%M'), alt.Tooltip('to', format='%Y-%m-%d %H:%M')],
            color=alt.Color('label', scale=alt.Scale(scheme='dark2'))
        ).properties(
            width=1200,
            height=200
        ).save('chart_from_chatbot.html')

    def toJSON(self):
        dct = deepcopy(self.__dict__)

        serializable_sub_datetimes = {lb: [] for lb in self.labels}
        for k, intv_list in self.sub_datetimes.items():
            for intv in intv_list:
                serializable_sub_datetimes[k].append((intv.start_datetime.strftime('%Y-%m-%d %H:%M'),
                                                      intv.end_datetime.strftime('%Y-%m-%d %H:%M')))

        dct["sub_datetimes"] = serializable_sub_datetimes
        
        ladder = self.current_dtrl["ladder"].toJSON()
        dct["current_dtrl"] = [self.current_dtrl["label"], ladder]

        return json.dumps(dct)

    @staticmethod
    def fromJSON(json_str):
        dct = json.loads(json_str)
        tt = TimeTable(dct['labels'])

        parsed_sub_datetimes = {lb: [] for lb in dct['labels']}
        for k, intv_list in dct['sub_datetimes'].items():
            for sd, ed in intv_list:
                parsed_sub_datetimes[k].append(DateTimeRange(sd, ed))

        tt.sub_datetimes = parsed_sub_datetimes
        
        tt.current_dtrl = {
            "label": dct["current_dtrl"][0],
            "ladder": dct["current_dtrl"][1]
        }
        tt.has_currently_discussed_range = dct["has_currently_discussed_range"]

        return tt


In [75]:
class DateRangeLadder:
    
    def __init__(self, top_range):
        self.top_range = top_range
        self.ladder = [top_range] # [smaller --> greater]
        
    def step_in_ladder(self, daterange):
        inserted = False
        for i in range(len(self.ladder)):
            intersection = self.ladder[i].intersection(daterange)
            if intersection.start_datetime and intersection.end_datetime:
                self.ladder = [intersection, *self.ladder[i:]]
                inserted = True
                
        if not inserted:
            self.ladder = []
            
    def step_in_ladder_with_text(self, query):
        if not has_date_mention(query):
            return None
        
        inserted = False
        for i in range(len(self.ladder)):
            current = self.ladder[i]
            success_flag, user_date_mentions = extract_datetime_within_interval(current.start_datetime,
                                                                                current.end_datetime,
                                                                                query,
                                                                                expect_future_day=True)
            dt_mention = user_date_mentions[0]
            mention_dtr = DateTimeRange(dt_mention['start_date'],
                                        dt_mention['end_date'])
            intersection = self.ladder[i].intersection(mention_dtr)
                            
            if intersection.start_datetime and intersection.end_datetime:
                self.ladder = [intersection, *self.ladder[i:]]
                inserted = True
                    
        if not inserted:
            self.ladder = []
            
    def get_bottom_step(self):
        if self.has_range():
            return self.ladder[0]
        else:
            return None
        
    def has_range(self):
        return bool(self.ladder)
            
    def toJSON(self):
        dct = {}
        dct['top_range'] = (self.top_range.start_datetime.strftime('%Y-%m-%d %H:%M'),
                            self.top_range.end_datetime.strftime('%Y-%m-%d %H:%M'))
        
        ladder_s = []
        for dtr in self.ladder:
            ladder_s.append((dtr.start_datetime.strftime('%Y-%m-%d %H:%M'),
                             dtr.end_datetime.strftime('%Y-%m-%d %H:%M')))

        dct["ladder"] = ladder_s

        return json.dumps(dct)

    @staticmethod
    def fromJSON(json_str):
        dct = json.loads(json_str)
        top_dtr_tup = dct['top_range']
        drl = DateRangeLadder(DateTimeRange(top_dtr_tup[0], top_dtr_tup[1]))

        ladder = []
        for dtr_dct in dct['ladder']:
            ladder.append(DateTimeRange(dtr_dct[0], dtr_dct[1]))

        drl.ladder = ladder

        return drl

In [84]:
class ActionBlocks:
    
    def __init__(text: str, time_table: TimeTable):
        self.text = text
        self.time_table = time_table
    
    def if_text_has_datetime(self):
        return has_date_mention(self.text)
    
    def if_theres_currently_discussed_range(self):
        return self.time_table.has_currently_discussed_range


In [76]:
tt = TimeTable(["bot_free", "user_free"])

In [77]:
tt.set_current_discussed(
    {'start_date': datetime(2021, 10, 26, 17, 0),
     'end_date': datetime(2021, 10, 26, 19, 0)},
    "user_free"
)

In [78]:
tt.discuss_current("17:15-kor")

'REQUEST_OK'

In [79]:
tt.get_currently_discussed_range()

2021-10-26T17:15:00 - 2021-10-26T17:15:59

In [80]:
tt.discuss_current("abcd")

'REQUEST_SKIPPED'

In [81]:
tt.get_currently_discussed_range()

2021-10-26T17:15:00 - 2021-10-26T17:15:59

In [82]:
tt.remove_currently_discussed()

In [83]:
tt.get_currently_discussed_range()

'REQUEST_SKIPPED'

In [106]:
tt1 = TimeTable.fromJSON(tt.toJSON())

In [107]:
tt1

In [83]:
a = DateTimeRange(datetime(2021, 10, 1), datetime(2021, 10, 10))
b = DateTimeRange(datetime(2021, 10, 3), datetime(2021, 10, 5))
c = DateTimeRange(datetime(2021, 10, 6), datetime(2021, 10, 8))
d = DateTimeRange(datetime(2021, 10, 20), datetime(2021, 10, 25))

In [84]:
dtrl = DateRangeLadder(a)

In [85]:
dtrl.step_in_ladder(b)
print(dtrl.ladder)
dtrl.step_in_ladder(c)
print(dtrl.ladder)
dtrl.step_in_ladder(d)
print(dtrl.ladder)


[2021-10-03T00:00:00 - 2021-10-05T00:00:00, 2021-10-01T00:00:00 - 2021-10-10T00:00:00]
[2021-10-06T00:00:00 - 2021-10-08T00:00:00, 2021-10-01T00:00:00 - 2021-10-10T00:00:00]
[]


In [86]:
a = DateTimeRange(datetime(2021, 10, 24), datetime(2021, 11, 4))

dtrl = DateRangeLadder(a)

In [87]:
dtrl.step_in_ladder_with_text("kedd")
dtrl.ladder

2021-10-26T00:00:00 - 2021-10-26T23:59:59


[2021-10-26T00:00:00 - 2021-10-26T23:59:59,
 2021-10-24T00:00:00 - 2021-11-04T00:00:00]

In [88]:
dtrl.step_in_ladder_with_text("délelőtt")
dtrl.ladder

2021-10-26T10:00:00 - 2021-10-26T11:59:59
2021-10-26T10:00:00 - 2021-10-26T11:59:59


[2021-10-26T10:00:00 - 2021-10-26T11:59:59,
 2021-10-26T00:00:00 - 2021-10-26T23:59:59,
 2021-10-24T00:00:00 - 2021-11-04T00:00:00]

In [89]:
dtrl.step_in_ladder_with_text("szerdán")
dtrl.ladder

2021-10-27T00:00:00 - 2021-10-27T23:59:59
2021-10-27T00:00:00 - 2021-10-27T23:59:59
2021-10-27T00:00:00 - 2021-10-27T23:59:59


[2021-10-27T00:00:00 - 2021-10-27T23:59:59,
 2021-10-24T00:00:00 - 2021-11-04T00:00:00]

In [90]:
dtrl_1 = DateRangeLadder.fromJSON(dtrl.toJSON())

In [92]:
dtrl_1.step_in_ladder_with_text("reggel")
dtrl_1.ladder

2021-10-27T06:00:00 - 2021-10-27T09:59:59
2021-10-27T06:00:00 - 2021-10-27T09:59:59


[2021-10-27T06:00:00 - 2021-10-27T09:59:59,
 2021-10-27T00:00:00 - 2021-10-27T23:59:00,
 2021-10-24T00:00:00 - 2021-11-04T00:00:00]

In [32]:
import hun_date_parser as hdp
hdp.__version__

AttributeError: module 'hun_date_parser' has no attribute '__version__'

In [6]:
from datetime import datetime

In [30]:
!pip install ./.libs/hun_date_parser-0.0.9rc2-py3-none-any.whl

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Processing ./.libs/hun_date_parser-0.0.9rc2-py3-none-any.whl
  Attempting uninstall: hun-date-parser
    Found existing installation: hun-date-parser 0.0.9rc1
    Uninstalling hun-date-parser-0.0.9rc1:
      Successfully uninstalled hun-date-parser-0.0.9rc1


In [8]:
def get_available_appointments():
    """
    Loads available appointment list.
    """
    now = datetime.now()  # Get the current date

    with open('test_data.json', 'r') as f:  # Opening test_data
        data = json.load(f)

    # Parsing the test_data into a dictionary
    res = []
    for day in data:
        parsed = {'start_date': datetime.fromisoformat(day['start_date']),
                  'end_date': datetime.fromisoformat(day['end_date'])}

        # Normalizing the data
        if parsed['end_date'] <= now:
            continue
        if parsed['start_date'] <= now:
            parsed['start_date'] = now
        res.append(parsed)

    return res

In [9]:
get_available_appointments()

[{'start_date': datetime.datetime(2021, 10, 21, 17, 0),
  'end_date': datetime.datetime(2021, 10, 21, 19, 0)},
 {'start_date': datetime.datetime(2021, 10, 26, 17, 0),
  'end_date': datetime.datetime(2021, 10, 26, 23, 0)},
 {'start_date': datetime.datetime(2021, 10, 29, 11, 0),
  'end_date': datetime.datetime(2021, 10, 29, 19, 0)},
 {'start_date': datetime.datetime(2021, 11, 1, 6, 0),
  'end_date': datetime.datetime(2021, 11, 1, 19, 0)}]

In [13]:
time_table = TimeTable(['user_free', 'bot_free'])
for rec in get_available_appointments():
    time_table.label_timerange(rec['start_date'], rec['end_date'], 'bot_free')

In [14]:
time_table.query_timerange(datetime(2021, 10, 11, 12, 0),
                        datetime(2021, 10, 11, 15, 0),
                        'bot_free')

[]

In [15]:
time_table.query_first_range_for_label('bot_free')

2021-10-17T09:00:00 - 2021-10-17T19:00:00

In [21]:
from datetime import datetime

In [20]:
from hun_date_parser.date_parser.interval_restriction import extract_datetime_within_interval

In [22]:
extract_datetime_within_interval(datetime(2021, 10, 14), datetime(2021, 10, 15), 'reggel')

(<ExtractWithinRangeSuccess.VALID_IN_RANGE: 'in_range'>,
 [{'start_date': datetime.datetime(2021, 10, 14, 6, 0),
   'end_date': datetime.datetime(2021, 10, 14, 9, 59, 59)}])

In [27]:
(datetime(2021, 10, 11, 5) - datetime(2021, 10, 11, 4)).seconds

3600

In [11]:
from hun_date_parser import text2datetime

In [12]:
text2datetime("szerda")

[{'start_date': datetime.datetime(2021, 10, 20, 0, 0),
  'end_date': datetime.datetime(2021, 10, 20, 23, 59, 59)}]

In [13]:
text2datetime("szerda", expect_future_day=True)

[{'start_date': datetime.datetime(2021, 10, 27, 0, 0),
  'end_date': datetime.datetime(2021, 10, 27, 23, 59, 59)}]